### Save song names associated with their artists

In [118]:
import re
import pickle

In [80]:
lines=list()
with open("backstage/genius-api-download-log.txt", "r") as data:
    for line in data:
        pure_line = line.strip()
        if pure_line:
            lines.append(pure_line)

In [119]:
with open ('data/498artists.txt', 'rb') as fp:
    list_1 = pickle.load(fp)  
    
d={artist: None for artist in list_1}
no_found_artist_list=list()
artist_order_list=list()
artist_tuple_list=list()
song_list=list()
number_list=list()

In [121]:
len(list_1)

548

In [82]:
for line in lines:
    # Searching for songs by Xxxx Xxxx Xxxx Xxxx Xxxx...
    if re.match(r'Searching for songs by .*\s?.*\s?.*\s?.*\s?.*\s?\.\.\.',line):
        list_element=re.match(r'Searching for songs by .*\s?.*\s?.*\s?.*\s?.*\s?\.\.\.',line).group(0)
        artist_name=re.match(r'Searching for songs by .*\s?.*\s?.*\s?.*\s?.*\s?\.\.\.',line).group(0)[23:-3]
        # Exception: 'No results found for Xxx' or 'Timeout raised and caught:'
        if re.match(r"No results found for \'.*\'\.",lines[lines.index(list_element)+1]) or re.match(r'Timeout raised and caught\:',lines[lines.index(list_element)+1]):
            no_found_artist_list.append(artist_name)
        else:
            artist_order_list.append(artist_name)
        
    # Find artist names that Genius API changed automatically to replace the scrapped artist names
    # Changing artist name to 'Xxxx Xxxx Xxxx Xxxx Xxxx'
    if re.match(r"Changing artist name to '.*\s?.*\s?.*\s?.*\s?.*\s?'",line):
        list_element=re.match(r"Changing artist name to '.*\s?.*\s?.*\s?.*\s?.*\s?'",line).group(0)
        artist_name=list_element[25:-1]
        line_where_artist_to_be_replaced=lines[lines.index(list_element)-1]
        # Searching for songs by Xxxx Xxxx Xxxx Xxxx Xxxx...
        if re.match(r'Searching for songs by .*\s?.*\s?.*\s?.*\s?.*\s?\.\.\.',line_where_artist_to_be_replaced):
            artist_name_to_be_replaced=re.match(r'Searching for songs by .*\s?.*\s?.*\s?.*\s?.*\s?\.\.\.',line_where_artist_to_be_replaced).group(0)[23:-3]
            # Update artist name (key) in dictionary to new artist name
            artist_tuple=(artist_name_to_be_replaced,artist_name)
            artist_tuple_list.append(artist_tuple)
            artist_tuple=()
            #d[artist_name]=d.pop(artist_name_to_be_replaced)      

    # Get number of song grabbed
    # don't use 'Songs grabbed:\d\d?', because 'some exception at Xxxx: 265', use 'Done. Found __ songs.'
    if re.match(r'Done\. Found \d\d? songs\.',line): 
        number=re.match(r'Done\. Found \d\d? songs\.',line).group(0)[12:-7]
        number_list.append(number)
            
    # Get names of song grabbed
    if re.match(r'Song \d: \".*\"',line):
        song_name_1_9=re.match(r'Song \d: \".*\"',line).group(0)[9:-1]
        song_list.append(song_name_1_9)
        
    if re.match(r'Song \d\d: \".*\"',line):
        song_name_11_20=re.match(r'Song \d\d: \".*\"',line).group(0)[10:-1]
        song_list.append(song_name_11_20)

# repeated download for certain artists !!!

In [112]:
c=0
for artist in artist_order_list:
    idx=artist_order_list.index(artist)
    if idx==0:
        d[artist]=(number_list[idx],song_list[0:int(number_list[idx])])
        c+=int(number_list[idx])

    else:
        d[artist]=(number_list[idx],song_list[c:c+int(number_list[idx])])
        c+=int(number_list[idx])

In [ ]:
with open('backstage/artist-song-titles.pickle', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('backstage/artist-song-titles.pickle', 'rb') as handle:
#    b = pickle.load(handle)

In [ ]:
no_found_artist_list

In [ ]:
artist_tuple_list